In [16]:
import tweepy


In [17]:
with open("D:/github/twitter.txt", encoding="utf8") as f:
    client = tweepy.Client(f.readline().strip())

In [18]:
keyword = "손흥민"
target = list(tweepy.Paginator(client.search_recent_tweets, keyword, tweet_fields=["created_at"], max_results=10, limit=1).flatten())
print(len(target))

10


In [ ]:
for i in target:
    print(type(i), i)